In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
ohe = OneHotEncoder(sparse=False)
labelenc = LabelEncoder()

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#import CSV to be read as a pandas dataframe
data = pd.read_csv('/content/drive/MyDrive/Preprocessed_BRCA_RNA_data_normalized.csv')

#data = pd.get_dummies(data, columns=['Class'])
data['Class'] = labelenc.fit_transform(data['Class'])
data['Sample-id'] = labelenc.fit_transform(data['Sample-id'])


print(data)

x = data

print(x.shape)


      Sample-id  Class  ?|100130426  ...  ZZZ3|26009  psiTPTE22|387590  tAKR|389932
0             0      2       0.0000  ...   1180.4565            1.7233       0.0000
1             1      2       0.0000  ...    406.7428          926.5905       0.0000
2             2      2       0.9066  ...    509.5195           35.3581       0.0000
3             3      2       0.0000  ...    700.8688           66.6115       0.0000
4             4      2       0.0000  ...    881.7021          187.2340       0.0000
...         ...    ...          ...  ...         ...               ...          ...
1090       1090      2       0.0000  ...    443.0869          724.9033       0.0000
1091       1091      2       0.0000  ...    670.0787           98.4252       0.0000
1092       1092      2       0.0000  ...    953.7081          235.2385       0.9447
1093       1093      2       0.0000  ...    750.8288          238.9272       0.0000
1094       1094      2       0.0000  ...    462.1140           20.8786      

In [5]:
#splitting data into test and training set
x_train, x_test = train_test_split(x, test_size=0.2)

print(x_train.shape)
print(x_test.shape)

#scale data & normalize
t = MinMaxScaler()
t.fit(x_train)
t.fit(x_test)
train = t.transform(x_train)
test = t.transform(x_test)

(876, 20533)
(219, 20533)


In [6]:
#preparing model for data input
enc_input = keras.Input(shape=(None, 20533), name = 'raw_data') 

#adding noise 
noise_input = tf.keras.layers.GaussianNoise(stddev=0.1)(enc_input)

#encoding layers
encoder = model = tf.keras.models.Sequential()
encoder.add(tf.keras.layers.Flatten())
encoder = keras.layers.Dense(units=20533, activation = 'relu')(noise_input)
encoder = keras.layers.Dense(units=15000, activation='relu')(encoder)
encoder = keras.layers.Dense(units=10000, activation='relu')(encoder)
bottleneck = Dense(units=8000, activation='relu')(encoder)

In [7]:
#decoding layers
decoder = keras.layers.Dense(units=10000, activation='relu')(bottleneck)
decoder = keras.layers.Dense(units=15000, activation='relu')(decoder)
decoder = keras.layers.Dense(units=20533, activation='sigmoid')(decoder)

In [8]:
#Saving into two portions
#Autoencoder must be fit first before removing encoder piece
autoencoder = Model(enc_input, decoder)
encoder = Model(enc_input, bottleneck)

#Change learning rate on optimizer to improve loss
opt = keras.optimizers.Adam(learning_rate=0.001)


In [9]:
autoencoder.compile(optimizer=opt, loss = 'mse' , metrics=['accuracy'])

In [10]:
#Training model
autoencoder.fit(train, train ,epochs = 500, batch_size=250, shuffle = True, validation_data=(test, test))

Epoch 1/500
4/4 [==============================] - 20s 4s/step - loss: 3.3993 - accuracy: 0.0000e+00 - val_loss: 0.1056 - val_accuracy: 0.8402
Epoch 2/500
4/4 [==============================] - 17s 4s/step - loss: 5.2855 - accuracy: 0.0000e+00 - val_loss: 0.0864 - val_accuracy: 0.8402
Epoch 3/500
4/4 [==============================] - 17s 4s/step - loss: 4.2334 - accuracy: 0.0000e+00 - val_loss: 0.0808 - val_accuracy: 0.0046
Epoch 4/500
4/4 [==============================] - 17s 4s/step - loss: 2.7496 - accuracy: 0.0000e+00 - val_loss: 0.0798 - val_accuracy: 0.0046
Epoch 5/500
4/4 [==============================] - 17s 4s/step - loss: 2.8216 - accuracy: 0.0000e+00 - val_loss: 0.0786 - val_accuracy: 0.0046
Epoch 6/500
4/4 [==============================] - 17s 4s/step - loss: 3.4709 - accuracy: 0.0000e+00 - val_loss: 0.0776 - val_accuracy: 0.0046
Epoch 7/500
4/4 [==============================] - 17s 4s/step - loss: 4.2966 - accuracy: 0.0000e+00 - val_loss: 0.0769 - val_accuracy: 0.0046

In [11]:
#Saving encoder piece for data compression/feature extraction
encoder = Model(inputs=enc_input, outputs = bottleneck)
encoder.save('SDAEencoder.h5')

In [12]:
#data is compressed using only the encoder portion of the SAE
encoded_data = encoder.predict(x)

In [13]:
#saves encoded data as CSV
np.savetxt('Encoded_SDAEdata.csv', encoded_data, delimiter=",")


In [14]:
#verifies that all samples are encoded, 4000 features
print(encoded_data.shape)

(1095, 8000)
